# 1. Scrape the Wikipedia page

In [1]:
#Scrape the table from Wikipedia page
%matplotlib inline
import pandas as pd
import numpy as np
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
tables = soup.find('table',{'class':'wikitable sortable'}).tbody #find the table

rows=tables.find_all('tr')
columns=[v.text.replace('\n','') for v in rows[0].find_all('th')] # find the column names

df=pd.DataFrame(columns=columns)

for i in range(1,len(rows)): 
    tds=rows[i].find_all('td')
    values=[tds[0].text.replace('\n',''),tds[1].text.replace('\n',''),tds[2].text.replace('\n','')]
    df=df.append(pd.Series(values,index=columns),ignore_index=True)


In [2]:
# Wrangle the data
df=df[df['Borough']!="Not assigned"]
df.shape
df.head()

,Post Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# 2. Get the coordinator of each neighborhood

In [3]:
#Download the data
import wget
url='http://cocl.us/Geospatial_data'
Geospatial_Coordinates= wget.download(url)
data = pd.read_csv(Geospatial_Coordinates) #coordinate data

#Merge two tables
df_corrdinated=df.merge(data,left_on='Post Code',right_on='Postal Code').drop('Postal Code',axis=1)
df_corrdinated.rename(columns={"Post Code":"PostalCode"}, inplace= True )
df_corrdinated.head()
neighborhoods=df_corrdinated.copy()

# 3. Explore and cluster the neighborhoods in Toronto

Download all the dependencies that we will need

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Define an instance of the geocoder, the user agent is toronto_explorer

In [5]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Canada Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Canada Toronto are 43.6534817, -79.3839347.


## 3.1 Create a map of Toronto with neighborhoods superimposed on top.

### 3.1.1 The whole neighborhoods

In [6]:
%matplotlib inline
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### 3.1.2 Toronto neighborhoods whoes name contains "Toronto"

Banana is an old-school person, he/she only prefer to the Borough whose name contains "Toronto".
So I decide to work with only boroughs that contain the word Toronto

In [7]:
Toronto_sub_data = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
%matplotlib inline
#Create map for the selected area
# create map of Toronto using latitude and longitude values
latitude = 43.653908
longitude = -79.384293
map_Toronto_sub = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_sub_data['Latitude'], Toronto_sub_data['Longitude'], Toronto_sub_data['Borough'], Toronto_sub_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto_sub)  
    
map_Toronto_sub

### 3.1.3 Definie Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'PATK1E03HKIFW5QPDDKW5JM0XUKV3LI0ILOCNSTPNBKUS5WC' # your Foursquare ID
CLIENT_SECRET = 'W5HKS1OU2KAMPYCYOY13E1SDDMDQHLFYQFGV0CHG25AVBVV5' # your Foursquare Secret
VERSION = '20200605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PATK1E03HKIFW5QPDDKW5JM0XUKV3LI0ILOCNSTPNBKUS5WC
CLIENT_SECRET:W5HKS1OU2KAMPYCYOY13E1SDDMDQHLFYQFGV0CHG25AVBVV5


### 3.2 Explore neighborhood

Explore the first neigborhood

In [9]:
Toronto_sub_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [10]:
neighborhood_latitude = Toronto_sub_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_sub_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_sub_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


 **Get the top 100 venues that are in Regent Park, Harbourfront within a radius of 1000 meters**

In [11]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius =2000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PATK1E03HKIFW5QPDDKW5JM0XUKV3LI0ILOCNSTPNBKUS5WC&client_secret=W5HKS1OU2KAMPYCYOY13E1SDDMDQHLFYQFGV0CHG25AVBVV5&v=20200605&ll=43.6542599,-79.3606359&radius=2000&limit=100'

In [12]:
results = requests.get(url).json()

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,The Distillery Historic District,Historic Site,43.650244,-79.359323
3,Distillery Sunday Market,Farmers Market,43.650075,-79.361832
4,Arvo,Coffee Shop,43.649963,-79.361442


**Explore all the neighborhoods in Toronto** 

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Toronto_venues = getNearbyVenues(names=Toronto_sub_data['Neighborhood'],
                                   latitudes=Toronto_sub_data['Latitude'],
                                   longitudes=Toronto_sub_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [17]:
# The size of resulting dataframe
print('The total venues in Toronto: ',Toronto_venues.shape)
Toronto_venues.head()

#Check how many venues at eaach neighborhood
Toronto_venues.groupby('Neighborhood').count()

#Check how many unique venue category
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

The total venues in Toronto:  (3875, 7)
There are 230 uniques categories.


## 3.3 Analyze each neigborhood

### 3.3.1 Generate Top Veunue dataset

In [18]:
# one hot encoding
Toronto_venues_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_venues_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_venues_onehot.columns[-1]] + list(Toronto_venues_onehot.columns[:-1])
Toronto_venues_onehot = Toronto_venues_onehot[fixed_columns]

Toronto_venues_onehot.head()


,Zoo,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Non-Profit,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
# Check the Toronto_venues_onehot dimension
Toronto_venues_onehot.shape

(3875, 230)

**Group rows by neigborhood and by taking mean of the frequency of occurence of each category**

In [20]:
Toronto_grouped = Toronto_venues_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Zoo,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Non-Profit,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.0,0.00,0.01,0.0,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.0,0.0,0.01,0.0,0.00,0.0,0.02,0.01,0.01,0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.0,0.01,0.01,0.0,0.00,0.00,0.00,0.0,0.0,0.04,0.0,0.0,0.00,0.0,0.0,0.00,0.01,0.0,0.0,0.0,0.00,0.01,0.00,0.09,0.00,0.00,0.00,0.01,0.0,0.00,0.00,0.01,0.00,0.0,0.0,0.0,0.01,0.01,0.02,0.0,0.02,0.0,0.01,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.0,0.0,0.03,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.01,0.00,0.00,0.0,0.0,0.01,0.0,0.00,0.0,0.01,0.00,0.0,0.01,0.00,0.0,0.0,0.01,0.01,0.02,0.01,0.00,0.00,0.0,0.01,0.0,0.00,0.06,0.00,0.0,0.01,0.0,0.00,0.0,0.0,0.01,0.03,0.00,0.00,0.00,0.01,0.00,0.0,0.02,0.00,0.01,0.00,0.0,0.00,0.01,0.0,0.01,0.00,0.01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.04,0.0,0.01,0.0,0.00,

**Print each neighborhood with top 5 most common venues**

In [21]:
num_top_venues = 10

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.06
2                 Café  0.04
3           Restaurant  0.04
4                 Park  0.04
5  Japanese Restaurant  0.03
6       Farmers Market  0.03
7                Plaza  0.03
8     Baseball Stadium  0.02
9         Liquor Store  0.02


----Brockton, Parkdale Village, Exhibition Place----
                     venue  freq
0              Coffee Shop  0.05
1               Restaurant  0.04
2                     Café  0.04
3                   Bakery  0.04
4                     Park  0.03
5         Asian Restaurant  0.03
6                Gift Shop  0.03
7              Supermarket  0.02
8           Soccer Stadium  0.02
9  New American Restaurant  0.02


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0          Coffee Shop  0.08
1                 Park  0.06
2              Brewery  0.05
3                 Café  0.05
4  

**Put the Top 10 Venues data into a pandas dataframe**

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Park,Restaurant,Café,Japanese Restaurant,Plaza,Farmers Market,Liquor Store,Theater
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Restaurant,Café,Bakery,Gift Shop,Park,Asian Restaurant,Japanese Restaurant,Juice Bar,Soccer Stadium
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Park,Brewery,Café,Italian Restaurant,Pizza Place,Bakery,American Restaurant,Indian Restaurant,Beach
3,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Coffee Shop,Italian Restaurant,Hotel,Gym,Scenic Lookout,Brewery,Café,Yoga Studio,Harbor / Marina
4,Central Bay Street,Café,Pizza Place,Coffee Shop,Restaurant,Park,Bookstore,Theater,Yoga Studio,Supermarket,Cosmetics Shop


### 3.3.2 Cluster neigborhood

Run K=5 Kmeans 

In [24]:
# set number of clusters
kclusters =5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 4, 0, 1, 2, 1, 3, 0, 0])

Create the dataframe to include clusters and top 10 venues

Other clustering method

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_sub_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Gastropub,Park,Restaurant,Café,Clothing Store,Bakery,Japanese Restaurant,Farmers Market,Liquor Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Restaurant,Park,Bookstore,Gastropub,Supermarket
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Gastropub,Japanese Restaurant,Café,Pizza Place,Park,Hotel,Clothing Store,Restaurant,Bubble Tea Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Hotel,Japanese Restaurant,Farmers Market,Theater,Clothing Store,Café,Restaurant,Plaza,Liquor Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Coffee Shop,Pub,Breakfast Spot,Beach,Japanese Restaurant,Bakery,Park,BBQ Joint,Ice Cream Shop,Middle Eastern Restaurant


**Visualize the resulting cluster**

In [26]:
# create map
%matplotlib inline
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.3.3 Examin clusters

**Cluster 1**

In [27]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Grocery Store,Park,Pub,Bakery,Pizza Place,Dog Run
19,Central Toronto,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Sushi Restaurant,Bakery,Middle Eastern Restaurant,Japanese Restaurant,Yoga Studio,Deli / Bodega
20,Central Toronto,0,Coffee Shop,Italian Restaurant,Park,Bakery,Café,Japanese Restaurant,Deli / Bodega,Indian Restaurant,Restaurant,Pizza Place
21,Central Toronto,0,Italian Restaurant,Coffee Shop,Café,Sushi Restaurant,Park,Yoga Studio,Bagel Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
23,Central Toronto,0,Coffee Shop,Italian Restaurant,Café,Bakery,Japanese Restaurant,Sushi Restaurant,Park,Fast Food Restaurant,Dessert Shop,Deli / Bodega
26,Central Toronto,0,Italian Restaurant,Café,Coffee Shop,Park,Restaurant,Bakery,Yoga Studio,Indian Restaurant,Sushi Restaurant,Japanese Restaurant
28,West Toronto,0,Park,Café,Bakery,Coffee Shop,Pub,Italian Restaurant,Sushi Restaurant,Gas Station,Grocery Store,Bar
29,Central Toronto,0,Park,Italian Restaurant,Coffee Shop,Sushi Restaurant,Bakery,Café,Restaurant,Indian Restaurant,Dessert Shop,Yoga Studio
31,Central Toronto,0,Italian Restaurant,Park,Coffee Shop,Café,Sushi Restaurant,Hotel,Grocery Store,Ice Cream Shop,French Restaurant,Yoga Studio
32,Downtown Toronto,0,Park,Coffee Shop,Italian Restaurant,Hotel,Gym,Scenic Lookout,Brewery,Café,Yoga Studio,Harbor / Marina


**Cluster 2**

In [28]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Restaurant,Park,Bookstore,Gastropub,Supermarket
2,Downtown Toronto,1,Coffee Shop,Gastropub,Japanese Restaurant,Café,Pizza Place,Park,Hotel,Clothing Store,Restaurant,Bubble Tea Shop
6,Downtown Toronto,1,Café,Pizza Place,Coffee Shop,Restaurant,Park,Bookstore,Theater,Yoga Studio,Supermarket,Cosmetics Shop
14,West Toronto,1,Coffee Shop,Restaurant,Café,Bakery,Gift Shop,Park,Asian Restaurant,Japanese Restaurant,Juice Bar,Soccer Stadium
24,Central Toronto,1,Coffee Shop,Café,Park,Restaurant,Grocery Store,Italian Restaurant,Ice Cream Shop,Korean Restaurant,Thai Restaurant,Japanese Restaurant
27,Downtown Toronto,1,Pizza Place,Café,Park,Coffee Shop,Mexican Restaurant,Sandwich Place,Beer Bar,Theater,Supermarket,Tapas Restaurant
30,Downtown Toronto,1,Pizza Place,Café,Beer Bar,Bakery,Italian Restaurant,Taco Place,Hotel,Sandwich Place,Park,Cocktail Bar
35,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Park,Bakery,Diner,Thai Restaurant,Japanese Restaurant,Dance Studio,Gastropub
37,Downtown Toronto,1,Coffee Shop,Café,Park,Japanese Restaurant,Gastropub,Hotel,Diner,Theater,Clothing Store,Restaurant


**Cluster 3**

In [29]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,2,Café,Coffee Shop,Bar,Italian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Beer Bar,Korean Restaurant,Ice Cream Shop
9,West Toronto,2,Café,Italian Restaurant,Park,Coffee Shop,Bar,Brewery,Cocktail Bar,Pizza Place,Indian Restaurant,Middle Eastern Restaurant
11,West Toronto,2,Café,Bakery,Pizza Place,Coffee Shop,Vietnamese Restaurant,Italian Restaurant,Restaurant,Cocktail Bar,Asian Restaurant,Athletics & Sports
12,East Toronto,2,Café,Park,Greek Restaurant,Vietnamese Restaurant,Bakery,Italian Restaurant,Coffee Shop,Pub,Ice Cream Shop,Yoga Studio
22,West Toronto,2,Café,Coffee Shop,Italian Restaurant,Brewery,Bakery,Bar,Gastropub,Pizza Place,Restaurant,Thai Restaurant
25,West Toronto,2,Café,Coffee Shop,Bakery,Italian Restaurant,Park,Bar,Restaurant,Eastern European Restaurant,Gastropub,Brewery


**Cluster 4**

In [30]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Gastropub,Park,Restaurant,Café,Clothing Store,Bakery,Japanese Restaurant,Farmers Market,Liquor Store
3,Downtown Toronto,3,Coffee Shop,Hotel,Japanese Restaurant,Farmers Market,Theater,Clothing Store,Café,Restaurant,Plaza,Liquor Store
5,Downtown Toronto,3,Coffee Shop,Hotel,Park,Restaurant,Café,Japanese Restaurant,Plaza,Farmers Market,Liquor Store,Theater
8,Downtown Toronto,3,Coffee Shop,Hotel,Café,Pizza Place,Sandwich Place,Restaurant,Plaza,Clothing Store,Theater,Japanese Restaurant
10,Downtown Toronto,3,Hotel,Café,Seafood Restaurant,Gym,Japanese Restaurant,Theater,Brewery,Plaza,Pizza Place,Coffee Shop
13,Downtown Toronto,3,Hotel,Coffee Shop,Plaza,Japanese Restaurant,Restaurant,Beer Bar,Clothing Store,Theater,Park,Pizza Place
16,Downtown Toronto,3,Hotel,Coffee Shop,Café,Japanese Restaurant,Clothing Store,Theater,Pizza Place,Plaza,Beer Bar,Restaurant
34,Downtown Toronto,3,Coffee Shop,Hotel,Café,Restaurant,Theater,Plaza,Farmers Market,Japanese Restaurant,Clothing Store,Park
36,Downtown Toronto,3,Coffee Shop,Hotel,Beer Bar,Theater,Clothing Store,Plaza,Café,Restaurant,Japanese Restaurant,Pizza Place


**Cluster 5**

In [31]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,4,Coffee Shop,Pub,Breakfast Spot,Beach,Japanese Restaurant,Bakery,Park,BBQ Joint,Ice Cream Shop,Middle Eastern Restaurant
15,East Toronto,4,Park,Coffee Shop,Brewery,Café,Beach,Bar,Indian Restaurant,Italian Restaurant,Pizza Place,American Restaurant
17,East Toronto,4,Coffee Shop,Park,Vietnamese Restaurant,Brewery,Bakery,Café,Bar,French Restaurant,Diner,Farmers Market
38,East Toronto,4,Coffee Shop,Park,Brewery,Café,Italian Restaurant,Pizza Place,Bakery,American Restaurant,Indian Restaurant,Beach
